In [72]:
import os, shutil, glob, requests, time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [73]:
os.makedirs(os.path.join("Real Property Dataset"),exist_ok = True)

In [74]:
def Scrape_Data(driver): 
    House_Information = []
    Page = driver.page_source
    Source = BeautifulSoup(Page, "lxml")

    Titles = Source.find_all("span", class_="pr-title js__card-title")
    Areas = Source.find_all("span", class_="re__card-config-area js__card-config-item")
    Bedrooms = Source.find_all("span", class_="re__card-config-bedroom js__card-config-item")
    Bathrooms = Source.find_all("span", class_="re__card-config-toilet js__card-config-item")
    Location = Source.find_all("div", class_="re__card-location")
    Prices = Source.find_all("span", class_="re__card-config-price js__card-config-item")
    
    for title, area, bedroom, bathroom, price, local in zip(Titles, Areas, Bedrooms, Bathrooms, Prices, Location):
        House_Information.append({
            'House Title': title.text.strip(),
            'Area': area.text.strip(),
            'Bedrooms': bedroom.find('span').text.strip(),
            'Bathrooms': bathroom.find('span').text.strip(),
            'Location': local.text.strip(),
            'Price': price.text.strip()
        })
    
    return House_Information

In [75]:

def Browser_Automation():
    driver = webdriver.Edge()
    driver.set_window_size(1280, 720)
    URL = "https://batdongsan.com.vn/ban-nha-rieng-tp-hcm?cIds=325,163"
    driver.get(URL)
    
    House_List = []

    while True:
        try:
            House_Data = Scrape_Data(driver)
            House_List.extend(House_Data)
            Next_Page = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.re__pagination-icon i.re__icon-chevron-right--sm")))
            Next_Page.click()
            time.sleep(1)                                   
        except Exception:
            break
            
    driver.quit()
    return House_List

House_List = pd.DataFrame(Browser_Automation())
House_List.to_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"), header=True, index=False)

In [76]:
List = pd.read_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"))
List

,House Title,Area,Bedrooms,Bathrooms,Location,Price
0,Lên sóng căn siêu phẩm ngay chợ HTT đường Lê V...,24 m²,2,2,"·\nGò Vấp, Hồ Chí Minh","3,2 tỷ"
1,Siêu phẩm đẹp say đắm trệt lửng 2 lầu 3PN có g...,50 m²,3,4,"·\nGò Vấp, Hồ Chí Minh","6,99 tỷ"
2,"Bán nhà phố 2 mặt view, ngay QL 13, Thủ Đức, g...",68 m²,4,6,"·\nThủ Đức, Hồ Chí Minh","6,4 tỷ"
3,"Nhà phố mini 3,45 tỷ Quang Trung P11 Gò Vấp đố...",28 m²,2,2,"Phường 1, Quận 3","3,45 tỷ"
4,Dinh thự ven sông biệt lập The Mizuki DT từ 53...,647 m²,5,7,"Phường 13, Phú Nhuận",63 tỷ
...,...,...,...,...,...,...
6862,"Cần gấp tiền học phí bán biệt thự Cao Thắng, Q...",150 m²,4,4,"Đa Kao, Quận 1","19,5 tỷ"
6863,"Bán nhà Huỳnh Tấn Phát, Q. 7 (8x25m) giá tốt 1...",173 m²,4,5,"Quận 7, Hồ Chí Minh","24,5 tỷ"
6864,"Bán nhà đường số Tân Kiểng (4,6x17m) 2 tầng, g...",77 m²,5,5,"Quận 10, Hồ Chí Minh","12,5 tỷ"
6865,"Bán nhà 1T2L mặt tiền đường Đỗ Xuân Hợp, Phước...",55 m²,4,5,"Quận 7, Hồ Chí Minh","14,5 tỷ"


In [100]:
List["House Title"] = List["House Title"].str.replace('"', "")
List["Area"] = List["Area"].str.replace("m²", "")
List["Location"] = List["Location"].str.replace("·\n", "")
List["Price"] = List["Price"].str.replace("tỷ", "").str.replace(",", ".")


In [101]:
List

,House Title,Area,Bedrooms,Bathrooms,Location,Price
0,Lên sóng căn siêu phẩm ngay chợ HTT đường Lê V...,24,2,2,"Gò Vấp, Hồ Chí Minh",3.2
1,Siêu phẩm đẹp say đắm trệt lửng 2 lầu 3PN có g...,50,3,4,"Gò Vấp, Hồ Chí Minh",6.99
2,"Bán nhà phố 2 mặt view, ngay QL 13, Thủ Đức, g...",68,4,6,"Thủ Đức, Hồ Chí Minh",6.4
3,"Nhà phố mini 3,45 tỷ Quang Trung P11 Gò Vấp đố...",28,2,2,"Phường 1, Quận 3",3.45
4,Dinh thự ven sông biệt lập The Mizuki DT từ 53...,647,5,7,"Phường 13, Phú Nhuận",63
...,...,...,...,...,...,...
6862,"Cần gấp tiền học phí bán biệt thự Cao Thắng, Q...",150,4,4,"Đa Kao, Quận 1",19.5
6863,"Bán nhà Huỳnh Tấn Phát, Q. 7 (8x25m) giá tốt 1...",173,4,5,"Quận 7, Hồ Chí Minh",24.5
6864,"Bán nhà đường số Tân Kiểng (4,6x17m) 2 tầng, g...",77,5,5,"Quận 10, Hồ Chí Minh",12.5
6865,"Bán nhà 1T2L mặt tiền đường Đỗ Xuân Hợp, Phước...",55,4,5,"Quận 7, Hồ Chí Minh",14.5


In [102]:
List.to_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"), header=True, index=False)